# Phân tích dữ liệu

## Download Data

In [ ]:
!wget https://www.dropbox.com/s/wa8d1sujzlx56hh/ETL_DATA_new.csv

--2024-07-20 06:53:48--  https://www.dropbox.com/s/wa8d1sujzlx56hh/ETL_DATA_new.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/xfqhfaqam1mbvmlapv5z1/ETL_DATA_new.csv?rlkey=fcox2uze73hrreid6e78nvufg [following]
--2024-07-20 06:53:48--  https://www.dropbox.com/scl/fi/xfqhfaqam1mbvmlapv5z1/ETL_DATA_new.csv?rlkey=fcox2uze73hrreid6e78nvufg
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccf8b9c20cb6e1fe15553577e9e.dl.dropboxusercontent.com/cd/0/inline/CXBfuj3OnsyakZZegAT0sfXupCPRJFrg0cXEIWz8SgPXujzug-KoOqb2bzpzv8bzgVLgvPOTJM-tD3ZQyCMnrUoZAmD0PxMZn6pJACJd5z5aE4J-dTzfmepkft0WLvzjKQkHlneaIGKpiFU8MbdSWoeW/file# [following]
--2024-07-20 06:53:48--  https://uccf8b9c20cb6e1fe15553577e9e.dl.dropboxusercontent.com/c

## Xử lý Data

In [ ]:
import pandas as pd
# Đọc dữ liệu
dataset = pd.read_csv('ETL_DATA_new.csv', index_col=0)
# Bỏ cột TimeStamp
dataset.drop('TimeStamp', inplace=True, axis=1)
# Bỏ cột Unnamed
sensorsWithPointInformation = [col for col in dataset if 'Unnamed' in col]
dataset.drop(columns=sensorsWithPointInformation, inplace=True, axis=1)
# Điền 0 cho giá trị null
dataset = dataset.fillna(0)

## Tạo Dataframe cho cảm biến có giá trị pm25 > 15

In [ ]:
thresholdValue = 15
pm25 = pd.DataFrame(columns=["long", "lat", "pm25"])
for col in dataset[1:]:
    res = [i for i in dataset[col].values if i >= thresholdValue]
    if len(res) == 0 or col == "Unnamed":
        dataset = dataset.drop([col], axis = 1)
    else:
        if "Poi" in col:
            #print("Hey")
            col = col.strip("Point()")
            col = col.rstrip(").1")
            long, lat = col.split()
            pm25 = pm25._append({'long': float(long), 'lat': float(lat), 'pm25': len(res)}, ignore_index=True)
pm25.head()

,long,lat,pm25
0,139.096021,36.404732,8204.0
1,139.042873,36.381603,8999.0
2,138.995512,36.338016,13929.0
3,139.342672,36.410566,12667.0
4,139.381732,36.290913,10391.0


## heatmap các cảm biến

In [ ]:
import pandas as pd
import plotly.express as px

fig = px.density_mapbox(pm25, lat = 'lat', lon = 'long', z = 'pm25',
                        radius = 8,
                        zoom = 6,
                        mapbox_style = 'open-street-map')
fig.show()

# Áp Dụng 3P_ECLAT

Cài đặt thư viện

In [ ]:
!pip install -U pami==2023.8.6.2

Đọc và chuyển đổi dữ liệu

In [ ]:
from PAMI.extras.DF2DB import denseDF2DB as db
obj = db.denseDF2DB(dataset, '>=', 35)
obj.createTemporal('temporalDatabasePM25HeavyPollution.csv')

Thông số và biểu đồ dữ liệu

In [ ]:
from PAMI.extras.dbStats import temporalDatabaseStats as tempDS
obj = tempDS.temporalDatabaseStats('temporalDatabasePM25HeavyPollution.csv')
obj.run()
obj.printStats()

Database size : 31822
Number of items : 1119
Minimum Transaction Size : 1
Average Transaction Size : 16.394318396078184
Maximum Transaction Size : 503
Minimum Inter Arrival Period : 1
Average Inter Arrival Period : 1.0
Maximum Inter Arrival Period : 1
Minimum periodicity : 593
Average periodicity : 5999.600536193029
Maximum periodicicty : 31775
Standard Deviation Transaction Size : 39.541170726391165
Variance : 1563.5533167645897
Sparsity : 0.9853491345879551


Mining Partial Periodic Patterns

In [ ]:
from PAMI.partialPeriodicPattern.basic import PPP_ECLAT as alg
obj = alg.PPP_ECLAT('temporalDatabasePM25HeavyPollution.csv',periodicSupport=500, period=1000, sep='\t')
obj.startMine()
obj.printResults()
obj.save('soramame_periodicFrequentPatterns.txt')

Partial Periodic Patterns were generated successfully using 3PEclat algorithm
Total number of Partial Periodic Patterns: 1087
Total Memory in USS: 1590595584
Total Memory in RSS 1612214272
Total ExecutionTime in ms: 10.898437976837158


Visualize các pattern được tìm thấy

In [ ]:
from PAMI.extras.graph import visualizePatterns as fig

obj = fig.visualizePatterns('soramame_periodicFrequentPatterns.txt',10)
obj.visualize(width=1000,height=900)

Number 	 Pattern
2	Point(133.7401624 34.5278135)	Point(133.4605487 34.4942837)	Point(133.7570366 34.5800143)
3	Point(133.7570366 34.5800143)	Point(133.7401624 34.5278135)	Point(133.8231236 34.6463042)
4	Point(133.7570366 34.5800143)	Point(133.7401624 34.5278135)	Point(133.7758944 34.6007931)
5	Point(130.8293822 33.8952617)	Point(130.8073796 33.9015062)	Point(130.8968264 33.9422417)
6	Point(130.8293822 33.8952617)	Point(130.8073796 33.9015062)	Point(130.7597913 33.8615931)
7	Point(135.4721574 34.5939635)	Point(135.4726219 34.6729017)	Point(135.4740101 34.6407367)
8	Point(130.6939898 33.89368080000001)	Point(130.8073796 33.9015062)	Point(130.7998865 33.868205)
9	Point(139.6623882 36.0656055)	Point(139.6681689 36.0701176)	Point(139.0432674 36.64710669999999)
10	Point(130.8293822 33.8952617)	Point(130.8073796 33.9015062)	Point(130.6939898 33.89368080000001)
11	Point(133.7570366 34.5800143)	Point(133.8362679 34.5899827)	Point(133.8171478 34.6075268)	Point(133.7758944 34.6007931)
